In [ ]:
import os
import cv2
import pafy
import math
import random
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf


In [ ]:
# seed_constant = 27
# np.random.seed(seed_constant)
# random.seed(seed_constant)

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                           mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                           mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
# Create a Matplotlib figure and specify the size of the figure.
plt.figure(figsize = (20, 20))

# Get the names of all classes/categories in UCF50.
all_classes_names = os.listdir('C:\\Users\\ritth\\code\\sign_data\\ten_frames')

# Generate a list of 20 random values. The values will be between 0-50, 
# where 50 is the total number of class in the dataset. 
random_range = random.sample(range(len(all_classes_names)), 1)

# Iterating through all the generated random values.
for counter, random_index in enumerate(random_range, 1):

    # Retrieve a Class Name using the Random Index.
    selected_class_Name = all_classes_names[random_index]

    # Retrieve the list of all the video files present in the randomly selected Class Directory.
    video_files_names_list = os.listdir(f'C:\\Users\\ritth\\code\\sign_data\\ten_frames\\{selected_class_Name}')

    # Randomly select a video file from the list retrieved from the randomly selected Class Directory.
    selected_video_file_name = random.choice(video_files_names_list)

    # Initialize a VideoCapture object to read from the video File.
    video_reader = cv2.VideoCapture(f'C:\\Users\\ritth\\code\\sign_data\\ten_frames\\{selected_class_Name}\\{selected_video_file_name}')
    
    # Read the first frame of the video file.
    _, bgr_frame = video_reader.read()

    # Release the VideoCapture object. 
    video_reader.release()

    # Convert the frame from BGR into RGB format. 
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

    # Write the class name on the video frame.
    cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Display the frame.
    plt.subplot(5, 4, counter);plt.imshow(rgb_frame);plt.axis('off')

In [ ]:
# Specify the height and width to which each video frame will be resized in our dataset.
#IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 30

# Specify the directory containing the UCF50 dataset. 
DATASET_DIR = "C:\\Users\\ritth\\code\\sign_data\\thirty_frames"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = np.array([ "mother", "hello", "like", "no", "meet"])

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh, pose])

In [ ]:
def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.doctor
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''

    # Declare a list to store video frames.
    frames_list = []
    
    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    #print(video_frames_count)

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)
    #skip_frames_window = 1
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Iterate through the Video Frames.
        for frame_counter in range(SEQUENCE_LENGTH):

            # Set the current frame position of the video.
            #if video_frames_count > 29:
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
            #else:
                #break

            # Reading the frame from the video. 
            success, frame = video_reader.read()

            image, results = mediapipe_detection(frame,holistic)
            draw_styled_landmarks(image, results)

            keypoints = extract_keypoints(results)

            # Check if Video frame is not successfully read then break the loop
            if not success:
                break
            
            # Resize the Frame to fixed height and width.
            #resized_frame = cv2.resize(frame, (200, 200))
            
            # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
            #normalized_frame = frame / 255

            # Append the normalized frame into the frames list
            frames_list.append(keypoints)
            #frames_list.append(frame)

    # Release the VideoCapture object. 
    video_reader.release()

    # Return the frames list.
    return frames_list
    

In [ ]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''

    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):
        
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.array(features)
    labels = np.array(labels)
    
    print(np.array(frames).shape)
    print(features.shape)
    
    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [ ]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

In [ ]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
print(features)
print(features.shape)

In [ ]:
# Split the Data into Train ( 75% ) and Test Set ( 25% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.25)

In [ ]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()

    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(SEQUENCE_LENGTH, 258)))
    #model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    #model.add(Dense(12, activation='relu'))                            
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [ ]:
# Construct the required LRCN model.
LRCN_model = create_LRCN_model()

In [ ]:

# Create an Instance of Early Stopping Callback.
#early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)

# Compile the model and specify loss function, optimizer and metrics to the model.
LRCN_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["categorical_accuracy"])

# numpy_array = np.array([[1,2,3],[3,4,5]])
# numpy_array = numpy_array.astype(float)
# numpy_array = tf.convert_to_tensor(numpy_array)

print(features_train.shape)
print(labels_train.shape)
#features_train = tf.convert_to_tensor(features_train)
labels_train = tf.convert_to_tensor(labels_train)

In [ ]:
# Start training the model.
LRCN_model_training_history = LRCN_model.fit(features_train, labels_train, epochs = 200, callbacks = [tb_callback])

In [ ]:
LRCN_model.save('actions2.h5')

In [ ]:
LRCN_model.load_weights('actions2.h5')

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = LRCN_model.predict(features_test)
ytrue = np.argmax( labels_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predicted_sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

       # Read the frame.
        ret, frame = cap.read() 

        image, results= mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[:30]
        
        #sequence.append(resized_frame)
        #sequence = sequence[-10:]
        

        #res = []
        #del(res)

        if len(sequence) == SEQUENCE_LENGTH:
            res = LRCN_model.predict(np.expand_dims(sequence, axis=0))[0]
            print(CLASSES_LIST[np.argmax(res)])
            
            sequence = []
            
            print(res[np.argmax(res)])
            
        #3. Viz logic
        #if 'res' in locals() or 'res' in globals():
        if res[np.argmax(res)] > threshold: 
            if len(sentence) > 0: 
                if CLASSES_LIST[np.argmax(res)] != sentence[-1]:
                    sentence.append(CLASSES_LIST[np.argmax(res)]) 
            else:
                sentence.append(CLASSES_LIST[np.argmax(res)])

        print(sentence)

        if len(sentence) > 5: 
            sentence = sentence[-5:]


        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        
        # if len(sentence) > 3:
        #    predicted_sentence = []
        #    predicted_sentence.append(nlp(sentence))
        #    sentence = [] 

        # elif len(sentence) == 2:
        #    cv2.putText(image, ' '.join(sentence), (3, 30), 
        #                cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.7, (255, 255, 255), 1, cv2.LINE_AA)
           
        # if len(predicted_sentence) != 0:
        #     cv2.putText(image, ' '.join(predicted_sentence), (3, 30), 
        #                cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.7, (255, 255, 255), 1, cv2.LINE_AA)

        cv2.putText(image, ' '.join(sentence), (3, 30), 
                        cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.7, (255, 255, 255), 1, cv2.LINE_AA)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
#cap.release()
#cv2.destroyAllWindows()    